In [1]:
# greetings to : https://github.com/PacktPublishing/Learn-OpenAI-Whisper/tree/main/Chapter07

# Learn OpenAI Whisper - Chapter 7

## Notebook 2: Quantizing Distil-Whisper with OpenVINO


![ch07_2-quantizing-distil-whisper-openvino.png](https://raw.githubusercontent.com/PacktPublishing/Learn-OpenAI-Whisper/main/Chapter07/ch07_2-quantizing-distil-whisper-openvino.png)

In this tutorial, we explore how to leverage the power of [Distil-Whisper](https://huggingface.co/distil-whisper/distil-large-v2), a distilled variant of OpenAI's  [Whisper](https://huggingface.co/openai/whisper-large-v2) model, in combination with OpenVINO for efficient and accurate automatic speech recognition (ASR). Distil-Whisper, proposed in the paper [Robust Knowledge Distillation via Large-Scale Pseudo Labelling](https://arxiv.org/abs/2311.00430), offers significant speed improvements and parameter reduction while maintaining comparable performance to the original Whisper model.

We delve into the architecture of Whisper, a Transformer-based encoder-decoder model that maps audio spectrogram features to text tokens. The process involves converting raw audio inputs to a log-Mel spectrogram, encoding the spectrogram using a Transformer encoder, and autoregressively predicting text tokens using a decoder.

To simplify the integration of Distil-Whisper with OpenVINO, we utilize the [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) library for loading the pre-trained model and the [Hugging Face Optimum](https://huggingface.co/docs/optimum) library for converting the model to OpenVINO™ IR format. Additionally, we apply `INT8` post-training quantization from [NNCF](https://github.com/openvinotoolkit/nncf/) to further improve the performance of the OpenVINO Distil-Whisper model.

This tutorial covers the following topics:

1. [Prerequisites](#Prerequisites): Setting up the necessary dependencies and environment.
2. [Loading PyTorch model](#Loading-PyTorch-model): Loading the Distil-Whisper model using PyTorch.
   - [Preparing input sample](#Preparing-input-sample): Preparing an audio input sample for inference.
   - [Running model inference](#Running-model-inference): Running inference on the PyTorch model.
3. [Loading OpenVINO model using Optimum library](#Loading-OpenVINO-model-using-Optimum-library): Converting the model to OpenVINO format using the Optimum library.
   - [Selecting Inference device](#Selecting-Inference-device): Choosing the appropriate inference device.
   - [Compiling OpenVINO model](#Compiling-OpenVINO-model): Compiling the OpenVINO model for optimal performance.
   - [Running OpenVINO model inference](#Running-OpenVINO-model-inference): Running inference on the OpenVINO model.
4. [Comparing performance PyTorch vs OpenVINO](#Comparing-performance-PyTorch-vs-OpenVINO): Benchmarking the performance of the PyTorch and OpenVINO models.
   - [Comparing with OpenAI Whisper](#Comparing-with-OpenAI-Whisper): Comparing Distil-Whisper with the original OpenAI Whisper model.
5. [Using OpenVINO model with HuggingFace pipelines](#Using-OpenVINO-model-with-HuggingFace-pipelines): Integrating the OpenVINO model with HuggingFace pipelines for seamless usage.
6. [Quantizing](#Quantizing): Applying post-training quantization to further optimize the model.
   - [Preparing calibration datasets](#Preparing-calibration-datasets): Preparing datasets for quantization calibration.
   - [Quantizing Distil-Whisper encoder and decoder models](#Quantizing-Distil-Whisper-encoder-and-decoder-models): Quantizing the encoder and decoder components of the Distil-Whisper model.
   - [Running quantized model inference](#Runnig-quantized-model-inference): Running inference on the quantized model.
   - [Comparing performance and accuracy of the original and quantized models](#Comparing-performance-and-accuracy-of-the-original-and-quantized-models): Evaluating the impact of quantization on performance and accuracy.
7. [Running interactive demo](#Running-interactive-demo): Exploring an interactive demonstration of the Distil-Whisper model.

By the end of this tutorial, you will have a comprehensive understanding of how to leverage Distil-Whisper and OpenVINO for efficient and accurate automatic speech recognition. Let's get started!

## Prerequisites

Before diving into the tutorial, ensure that you have the necessary prerequisites in place. This includes authenticating with the Hugging Face Hub using your token and verifying the authentication by running the provided code cells. These steps are crucial for accessing the required models and datasets throughout the tutorial.



In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Verify authentication
from huggingface_hub import whoami
whoami()
# you should see something like {'type': 'user',  'id': '...',  'name': 'Wauplin', ...}

In [ ]:
#aa: needed to run twice the following cell
%pip -q install sentence-transformers==2.3.1
%pip -q install "transformers==4.37.2"
%pip -q install onnx "git+https://github.com/huggingface/optimum-intel.git" "peft==0.6.2" --extra-index-url https://download.pytorch.org/whl/cpu
%pip -q install "openvino>=2023.2.0" datasets  "gradio>=4.0" "librosa" "soundfile"
%pip -q install "nncf>=2.6.0" "jiwer"

## Loading the PyTorch model

Loading the PyTorch Whisper model is a straightforward process using the transformers library. The `AutoModelForSpeechSeq2Seq.from_pretrained` method is employed to initialize the model. In this tutorial, we will use the `distil-whisper/distil-large-v2` model as the default example. Please note that the model will be downloaded during the first run, which may take some time.

However, you have the flexibility to choose from a variety of models available in the [Distil-Whisper Hugging Face collection](https://huggingface.co/collections/distil-whisper/distil-whisper-models-65411987e6727569748d2eb6). Some alternative options include `distil-whisper/distil-medium.en` and `distil-whisper/distil-small.en`. Additionally, models of the original Whisper architecture are also accessible, which you can explore further [here](https://huggingface.co/openai).

It's important to highlight the significance of preprocessing and post-processing in the model's usage. The `AutoProcessor` class, specifically the `WhisperProcessor`, plays a crucial role in preparing the audio input data for the model. It handles tasks such as converting the audio to a Mel-spectrogram representation and decoding the predicted output token IDs back into a string using the tokenizer.

To ensure a smooth and efficient workflow, the `AutoProcessor` class streamlines the preprocessing and post-processing steps, allowing you to focus on the core functionality of the Whisper model. By leveraging this class, you can easily integrate the Whisper model into your speech recognition pipeline, regardless of the specific model variant you choose.

In [8]:
import ipywidgets as widgets

model_ids = {
    "Distil-Whisper": [
        "distil-whisper/distil-large-v2",
        "distil-whisper/distil-medium.en",
        "distil-whisper/distil-small.en"
    ],
    "Whisper": [
        "openai/whisper-large-v3",
        "openai/whisper-large-v2",
        "openai/whisper-large",
        "openai/whisper-medium",
        "openai/whisper-small",
        "openai/whisper-base",
        "openai/whisper-tiny",
        "openai/whisper-medium.en",
        "openai/whisper-small.en",
        "openai/whisper-base.en",
        "openai/whisper-tiny.en",
    ]
}

model_type = widgets.Dropdown(
    options=model_ids.keys(),
    value="Distil-Whisper",
    description="Model type:",
    disabled=False,
)

model_type

Dropdown(description='Model type:', options=('Distil-Whisper', 'Whisper'), value='Distil-Whisper')

In [9]:
model_id = widgets.Dropdown(
    options=model_ids[model_type.value],
    value=model_ids[model_type.value][1],
    description="Model:",
    disabled=False,
)

model_id

Dropdown(description='Model:', index=1, options=('distil-whisper/distil-large-v2', 'distil-whisper/distil-medi…

In [10]:
!ls -alh  ~/.cache/huggingface/

total 8.0K
drwxr-xr-x. 2 1000800000 root 40 Jan 13 11:09 .
drwxr-xr-x. 6 1000800000 root 61 Jan 13 11:09 ..
-rw-r--r--. 1 1000800000 root 61 Jan 13 11:09 stored_tokens
-rw-r--r--. 1 1000800000 root 37 Jan 13 11:09 token


In [11]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained(model_id.value)

pt_model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id.value)
pt_model.eval();

/opt/app-root/lib64/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

/opt/app-root/lib64/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/789M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

In [13]:
!ls -alh  ~/.cache/huggingface/hub

total 4.0K
drwxr-xr-x. 4 1000800000 root 87 Jan 13 11:12 .
drwxr-xr-x. 3 1000800000 root 51 Jan 13 11:12 ..
drwxr-xr-x. 3 1000800000 root 54 Jan 13 11:12 .locks
drwxr-xr-x. 6 1000800000 root 65 Jan 13 11:12 models--distil-whisper--distil-medium.en
-rw-r--r--. 1 1000800000 root  1 Jan 13 11:12 version.txt


### Preparing the Input Sample

To use the Whisper model for speech recognition, we need to properly prepare the input audio sample. The `WhisperProcessor` expects the audio data to be in the form of a NumPy array, along with information about the audio sampling rate. It then processes the audio and returns the `input_features` tensor, which is used for making predictions.

The conversion of the audio file to the required NumPy format is conveniently handled by the Hugging Face Datasets library. This library provides a seamless interface for loading and preprocessing audio data, making it easier to integrate with the Whisper model.

To prepare the input sample, the next Python code:

1. Loads the audio file using the Hugging Face Datasets library.
2. Extracts the audio data as a NumPy array and obtain the sampling rate.
3. Passes the audio array and sampling rate to the `WhisperProcessor`.
4. Retrieves the `input_features` tensor from the processor.

In [14]:
!ls -lah ~/.cache/huggingface/datasets

ls: cannot access '/opt/app-root/src/.cache/huggingface/datasets': No such file or directory


In [15]:
from datasets import load_dataset

def extract_input_features(sample):
    input_features = processor(
        sample["audio"]["array"],
        sampling_rate=sample["audio"]["sampling_rate"],
        return_tensors="pt",
    ).input_features
    return input_features

dataset = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy", "clean", split="validation"
)
sample = dataset[0]
input_features = extract_input_features(sample)

README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/9.19M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/73 [00:00<?, ? examples/s]

In [16]:
!ls -lah ~/.cache/huggingface/datasets

total 4.0K
drwxr-xr-x. 3 1000800000 root 4.0K Jan 13 11:14 .
drwxr-xr-x. 4 1000800000 root   67 Jan 13 11:14 ..
drwxr-xr-x. 3 1000800000 root   19 Jan 13 11:14 hf-internal-testing___librispeech_asr_dummy
-rw-r--r--. 1 1000800000 root    0 Jan 13 11:14 _opt_app-root_src_.cache_huggingface_datasets_hf-internal-testing___librispeech_asr_dummy_clean_0.0.0_5be91486e11a2d616f4ec5db8d3fd248585ac07a.lock


In [18]:
#input_features

### Running Model Inference

With the input sample prepared, we can now perform speech recognition using the Whisper model. The model provides a convenient `generate` interface that simplifies the inference process. Here's how you can run the model inference:

1. Pass the `input_features` tensor to the `generate` method of the Whisper model.
2. The model will process the input and generate the predicted token IDs.
3. Once the generation is complete, use the `processor.batch_decode` method to decode the predicted token IDs into human-readable text transcription.

The `generate` method handles the complex task of sequence generation, taking into account the model's architecture and the provided input features. It produces the predicted token IDs, which represent the transcribed text in a encoded format.

By leveraging the `generate` interface and the `processor.batch_decode` method, you can easily perform speech recognition with the Whisper model. The model takes care of the complex task of mapping the audio input to text output, while the processor handles the necessary decoding step to provide you with the final transcription.

In [20]:
%%time
#import IPython.display as ipd

predicted_ids = pt_model.generate(input_features)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

#display(ipd.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"]))
print(f"Reference: {sample['text']}")
print(f"Result: {transcription[0]}")

Reference: MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL
Result:  Mr Quilter is the Apostle of the Middle Classes and we are glad to welcome his Gospel.
CPU times: user 25.2 s, sys: 5.01 s, total: 30.2 s
Wall time: 5.07 s


In [24]:
predicted_ids

tensor([[50257, 50362,  1770,  2264,   346,   353,   318,   262, 29748,   286,
           262,  6046, 38884,   290,   356,   389,  9675,   284,  7062,   465,
         23244,    13, 50256]])

In [25]:
transcription

[' Mr Quilter is the Apostle of the Middle Classes and we are glad to welcome his Gospel.']

## Loading OpenVINO Model using Optimum Library

To further optimize the performance of the Whisper model, we can leverage the Hugging Face Optimum API. This high-level API enables us to convert and quantize models from the Hugging Face Transformers library to the OpenVINO™ IR format, resulting in faster inference and reduced memory footprint. For more details, refer to the [Hugging Face Optimum documentation](https://huggingface.co/docs/optimum/intel/inference).

The Optimum Intel library provides a seamless way to load optimized models from the [Hugging Face Hub](https://huggingface.co/docs/optimum/intel/hf.co/models) and create pipelines for running inference with the OpenVINO Runtime using Hugging Face APIs. The Optimum Inference models are API-compatible with Hugging Face Transformers models, which means we only need to replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class.

To initialize the model class, we call the `from_pretrained` method. When downloading and converting the Transformers model, it's important to add the `export=True` parameter to ensure the model is properly converted to the OpenVINO format. After conversion, we can save the model for future use with the `save_pretrained` method.

One advantage of using the Optimum library is that the tokenizers and processors distributed with the models are also compatible with the OpenVINO model. This means we can reuse the previously initialized processor without any modifications.


In [29]:
from pathlib import Path
Path(model_id.value.replace('/', '_'))
Path(model_id.value)

PosixPath('distil-whisper/distil-medium.en')

In [30]:
!ls -alh  ~/.cache/huggingface/hub

total 4.0K
drwxr-xr-x. 5 1000800000 root 147 Jan 13 11:14 .
drwxr-xr-x. 4 1000800000 root  67 Jan 13 11:14 ..
drwxr-xr-x. 6 1000800000 root  65 Jan 13 11:14 datasets--hf-internal-testing--librispeech_asr_dummy
drwxr-xr-x. 4 1000800000 root 114 Jan 13 11:14 .locks
drwxr-xr-x. 6 1000800000 root  65 Jan 13 11:12 models--distil-whisper--distil-medium.en
-rw-r--r--. 1 1000800000 root   1 Jan 13 11:12 version.txt


In [31]:
from pathlib import Path
from optimum.intel.openvino import OVModelForSpeechSeq2Seq

model_path = Path(model_id.value.replace('/', '_'))
ov_config = {"CACHE_DIR": ""}

if not model_path.exists():
    ov_model = OVModelForSpeechSeq2Seq.from_pretrained(
        model_id.value, ov_config=ov_config, export=True, compile=False, load_in_8bit=False
    )
    ov_model.half()
    ov_model.save_pretrained(model_path)
else:
    ov_model = OVModelForSpeechSeq2Seq.from_pretrained(
        model_path, ov_config=ov_config, compile=False
    )

/opt/app-root/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 357, 366, 438, 532, 685, 705, 796, 930, 1058, 1220, 12

In [32]:
!ls -alh  ~/.cache/huggingface/hub

total 4.0K
drwxr-xr-x. 5 1000800000 root 147 Jan 13 11:14 .
drwxr-xr-x. 4 1000800000 root  67 Jan 13 11:14 ..
drwxr-xr-x. 6 1000800000 root  65 Jan 13 11:14 datasets--hf-internal-testing--librispeech_asr_dummy
drwxr-xr-x. 4 1000800000 root 114 Jan 13 11:14 .locks
drwxr-xr-x. 6 1000800000 root  65 Jan 13 11:12 models--distil-whisper--distil-medium.en
-rw-r--r--. 1 1000800000 root   1 Jan 13 11:12 version.txt


### Selecting the Inference Device

When running inference with the OpenVINO runtime, it's important to select the appropriate inference device based on your hardware capabilities and performance requirements. The OpenVINO toolkit supports a wide range of devices, including CPUs, GPUs, and specialized accelerators.

To select the inference device, you can use the `ov.Core` class from the `openvino` package. This class provides methods to query the available devices and set the desired device for inference.


In [33]:
import openvino as ov
import ipywidgets as widgets

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="AUTO",
    description="Device:",
    disabled=False,
)

device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

### Compiling the OpenVINO Model

After selecting the inference device, the next step is to compile the OpenVINO model for optimal performance on the chosen device. Compiling the model involves applying device-specific optimizations and generating an executable representation of the model that can be efficiently run on the target device.

To compile the OpenVINO model, you can use the `compile` method of the `OVModelForSpeechSeq2Seq` class.


In [34]:
ov_model.to(device.value)
ov_model.compile()

### Running Inference with the OpenVINO Model

With the OpenVINO model compiled and optimized for the selected inference device, we can now run inference to perform speech recognition. The process of running inference with the OpenVINO model is similar to the PyTorch model, but with the added benefits of improved performance and efficiency.

To run inference with the OpenVINO model, we follow these steps:

1. Prepare the input sample by extracting the audio features using the `WhisperProcessor`, as explained in the [Preparing input sample](#Preparing-input-sample) section.

2. Pass the input features to the `generate` method of the OpenVINO model, just like we did with the PyTorch model:

   ```python
   predicted_ids = ov_model.generate(input_features)
   ```

   The `generate` method takes the input features and performs the necessary computations using the optimized OpenVINO model to generate the predicted token IDs.

3. Decode the predicted token IDs into human-readable text using the `processor.batch_decode` method:

   ```python
   transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
   ```

   This step converts the predicted token IDs back into the corresponding text transcription, just like we did with the PyTorch model.


In [36]:
%%time
predicted_ids = ov_model.generate(input_features)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

#display(ipd.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"]))
print(f"Reference: {sample['text']}")
print(f"Result: {transcription[0]}")

Reference: MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL
Result:  Mr Quilter is the Apostle of the Middle Classes and we are glad to welcome his Gospel.
CPU times: user 15.2 s, sys: 1.46 s, total: 16.7 s
Wall time: 2.8 s


## Comparing Performance: PyTorch vs OpenVINO

Now that we have successfully run inference with both the PyTorch and OpenVINO models, it's crucial to compare their performance to understand the benefits of using OpenVINO optimization. Performance comparison helps us evaluate the speed and efficiency improvements achieved by converting the model to the OpenVINO format and utilizing the optimized inference runtime.

To compare the performance of the PyTorch and OpenVINO models, we can measure and analyze the following metrics:

1. **Inference Time**: Measure the time taken by each model to process an input audio sample and generate the transcription. This includes the time spent on preprocessing, inference, and postprocessing steps. By comparing the inference times, we can determine which model offers faster execution.

2. **Throughput**: Evaluate the number of audio samples that each model can process per unit of time. Higher throughput indicates better efficiency and the ability to handle a larger volume of audio data in a given timeframe. OpenVINO optimization aims to improve throughput by leveraging hardware-specific optimizations and efficient resource utilization.

3. **Memory Usage**: Monitor the memory consumption of each model during inference. Efficient memory management is crucial, especially when dealing with limited resources or running inference on edge devices. OpenVINO optimization techniques, such as model compression and quantization, can help reduce memory usage without significant impact on accuracy.

4. **CPU and GPU Utilization**: Analyze the utilization of CPU and GPU resources during inference for both models. OpenVINO optimization aims to maximize the utilization of available hardware resources, enabling efficient parallel processing and reducing idle time. By comparing resource utilization, we can assess how effectively each model harnesses the capabilities of the underlying hardware.

To perform the performance comparison, you can use the `measure_perf` function provided in the notebook. This function takes the model and input sample as arguments and measures the inference time over a specified number of iterations. It provides a reliable estimate of the model's performance.

```python
perf_torch = measure_perf(pt_model, sample)
perf_ov = measure_perf(ov_model, sample)
```

By comparing the performance metrics between the PyTorch and OpenVINO models, you can quantify the speed and efficiency improvements achieved through OpenVINO optimization. This comparison highlights the benefits of using OpenVINO for accelerating inference and making the most of the available hardware resources.



In [37]:
import time
import numpy as np
from tqdm.notebook import tqdm


def measure_perf(model, sample, n=10):
    timers = []
    input_features = extract_input_features(sample)
    for _ in tqdm(range(n), desc="Measuring performance"):
        start = time.perf_counter()
        model.generate(input_features)
        end = time.perf_counter()
        timers.append(end - start)
    return np.median(timers)

In [38]:
perf_torch = measure_perf(pt_model, sample)
perf_ov = measure_perf(ov_model, sample)

Measuring performance:   0%|          | 0/10 [00:00<?, ?it/s]

Measuring performance:   0%|          | 0/10 [00:00<?, ?it/s]

In [39]:
print(f"Mean torch {model_id.value} generation time: {perf_torch:.3f}s")
print(f"Mean openvino {model_id.value} generation time: {perf_ov:.3f}s")
print(f"Performance {model_id.value} openvino speedup: {perf_torch / perf_ov:.3f}")

Mean torch distil-whisper/distil-medium.en generation time: 4.831s
Mean openvino distil-whisper/distil-medium.en generation time: 2.997s
Performance distil-whisper/distil-medium.en openvino speedup: 1.612


## Integrating the OpenVINO Model with Hugging Face Pipelines

One of the key advantages of using the OpenVINO model is its seamless compatibility with the Hugging Face [pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.AutomaticSpeechRecognitionPipeline) interface for `automatic-speech-recognition`. This compatibility allows us to leverage the powerful features and utilities provided by the Hugging Face ecosystem while benefiting from the performance optimizations of OpenVINO.

The `pipeline` interface is particularly useful for transcribing long audio files. Distil-Whisper employs a chunked algorithm that efficiently processes long-form audio by breaking it down into smaller segments. This chunked approach has been shown to be 9 times faster than the sequential algorithm proposed in the original OpenAI Whisper paper. To enable chunking, you can pass the `chunk_length_s` parameter to the pipeline, specifying the desired chunk length in seconds. For Distil-Whisper, a chunk length of 15 seconds has been found to be optimal.

In the next Python code cells, we create a pipeline for automatic speech recognition using the OpenVINO model (`ov_model`). We set the `generation_config` of the OpenVINO model to match that of the PyTorch model to ensure consistent behavior. The `tokenizer` and `feature_extractor` from the `processor` are also passed to the pipeline to handle the necessary preprocessing and postprocessing steps.

By specifying `max_new_tokens`, we limit the maximum number of tokens generated during transcription. The `chunk_length_s` parameter is set to 15 seconds to enable efficient chunking of long audio files. Additionally, we can activate batching by passing the `batch_size` argument, which allows the pipeline to process multiple audio samples concurrently.

Using the pipeline with the OpenVINO model offers several benefits:

1. **Simplified API**: The pipeline provides a high-level and intuitive interface for performing automatic speech recognition, abstracting away the complexities of the underlying model and preprocessing steps.

2. **Long Audio Support**: With the chunked algorithm and the ability to specify the chunk length, the pipeline can efficiently handle long audio files, making it suitable for various real-world scenarios.

3. **Batching**: By enabling batching through the `batch_size` parameter, the pipeline can process multiple audio samples simultaneously, improving overall throughput and utilization of hardware resources.

4. **Performance Optimization**: The OpenVINO model, being optimized for inference, delivers faster and more efficient transcription compared to the PyTorch model, while maintaining comparable accuracy.

By leveraging the Hugging Face pipeline with the OpenVINO model, you can easily integrate state-of-the-art speech recognition capabilities into your applications, benefiting from the ease of use, flexibility, and performance optimizations provided by this powerful combination.


In [40]:
from transformers import pipeline

ov_model.generation_config = pt_model.generation_config

pipe = pipeline(
    "automatic-speech-recognition",
    model=ov_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
)

In [41]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample_long = dataset[0]


def format_timestamp(seconds: float):
    """
    format time in srt-file expected format
    """
    assert seconds >= 0, "non-negative timestamp expected"
    milliseconds = round(seconds * 1000.0)

    hours = milliseconds // 3_600_000
    milliseconds -= hours * 3_600_000

    minutes = milliseconds // 60_000
    milliseconds -= minutes * 60_000

    seconds = milliseconds // 1_000
    milliseconds -= seconds * 1_000

    return (
        f"{hours}:" if hours > 0 else "00:"
    ) + f"{minutes:02d}:{seconds:02d},{milliseconds:03d}"


def prepare_srt(transcription):
    """
    Format transcription into srt file format
    """
    segment_lines = []
    for idx, segment in enumerate(transcription["chunks"]):
        segment_lines.append(str(idx + 1) + "\n")
        timestamps = segment["timestamp"]
        time_start = format_timestamp(timestamps[0])
        time_end = format_timestamp(timestamps[1])
        time_str = f"{time_start} --> {time_end}\n"
        segment_lines.append(time_str)
        segment_lines.append(segment["text"] + "\n\n")
    return segment_lines

README.md:   0%|          | 0.00/480 [00:00<?, ?B/s]

(…)-00000-of-00001-913508124a40cb97.parquet:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

### Obtaining Timestamps and Generating Subtitles
The `pipeline` interface provides a convenient way to obtain timestamps associated with each processed chunk of audio. By passing the `return_timestamps` argument to the pipeline, you can retrieve the start and end timestamps of the speech segments corresponding to each chunk.

Timestamps can be incredibly useful in various scenarios, such as:

1. **Speech Separation**: If you have an audio file containing multiple speakers, the timestamps can help you identify and separate the speech segments corresponding to each speaker. This information can be used to create speaker-specific transcriptions or to isolate individual speakers for further analysis.

2. **Video Subtitles**: Timestamps play a crucial role in generating subtitles for videos. By aligning the transcribed text with the appropriate timestamps, you can create synchronized subtitles that match the speech in the video. This enhances the accessibility and comprehension of the video content.

In the provided example, we demonstrate how to format the transcription output in the popular SRT (SubRip Text) format, which is widely used for video subtitles. The SRT format consists of four parts for each subtitle:

1. Subtitle number
2. Start and end timestamps
3. Subtitle text
4. Blank line

In [42]:
result = pipe(sample_long["audio"].copy(), return_timestamps=True)

In [44]:
srt_lines = prepare_srt(result)

#display(
#    ipd.Audio(sample_long["audio"]["array"], rate=sample_long["audio"]["sampling_rate"])
#)
print("".join(srt_lines))

1
00:00:00,000 --> 00:00:06,600
 Mr Quilter is the Apostle of the Middle Classes, and we are glad to welcome his Gospel.

2
00:00:06,600 --> 00:00:11,320
 Nor is Mr Quilter's manner less interesting than his matter.

3
00:00:11,320 --> 00:00:18,000
 He tells us that at this festive season of the year with Christmas and roast beef looming before us,

4
00:00:18,000 --> 00:00:23,760
 similes drawn from eating and its results occur most readily to the mind.

5
00:00:23,760 --> 00:00:29,560
 He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can

6
00:00:29,560 --> 00:00:33,000
 discover in it but little of Rocky Ithaca.

7
00:00:33,000 --> 00:00:38,000
 Linnell's pictures are a sort of Up Guards and Adam paintings,

8
00:00:38,000 --> 00:00:43,000
 and Mason's exquisite itels are as national as a Jingo poem.

9
00:00:44,000 --> 00:00:49,200
 Mr. Birkett Foster's landscapes smile at one much in the same way that Mr.

10
00:00:49,200 --> 00:00:52,000
 K

## Interactive Demo: Experience Distil-Whisper in Action
...

In [46]:
from transformers.pipelines.audio_utils import ffmpeg_read
import gradio as gr
import urllib.request

urllib.request.urlretrieve(
    url="https://huggingface.co/spaces/distil-whisper/whisper-vs-distil-whisper/resolve/main/assets/example_1.wav",
    filename="example_1.wav",
)



('example_1.wav', <http.client.HTTPMessage at 0x7fa5696de070>)